# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
               
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_space
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
session.set_keyspace('sparkify_space')

In [8]:
#Drop Key Space
#session.execute(""" DROP KEYSPACE sparkify_space """)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


### The table structure for Query 1:

Only need two identificators for this table: sessionId, itemInSession & output of the query demands informations as : artist, song, song_length.
To model for these specifications, I will partition table with the sessionId and cluster by itemInSession.
* itemInSession - is a column with repeated values indicating that my clustering have optimum performace.

In [9]:
query = "CREATE TABLE IF NOT EXISTS sessions_table"
query = query + "(sessionId int, itemInSession int, artist text, song text, length decimal, PRIMARY KEY(sessionId, itemInSession))"
try: 
    session.execute(query)
except Exception as e:
    print(e)           

### Inserting Data into Table:

As the base data is of string type basic transformations were applied while inserting these 3 fields:
* sessionId should be an int
* itemInSession should be an int
* length should be a decimal

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO sessions_table (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Verifying result output from Query 1

In [12]:
query = "SELECT artist, song, length from sessions_table WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### The table structure for Query 2:

There are two identificators: userid and sessionId. 
They should act as a Composite Partiotion Key and itemInSession will act as cluster by (to order results).
Further the query should fetch the following fields: artist, song, first, last.

In [14]:
query = "CREATE TABLE IF NOT EXISTS user_lib"
query = query + "(userid int, sessionId int, itemInSession int, artist text, song text, first text, last text, PRIMARY KEY((userid, sessionId), itemInSession))"
try: 
    session.execute(query)
except Exception as e:
    print(e)    

### Inserting Data into Table:

As the base data is of string type basic transformations were applied while inserting these 3 fields:
* sessionId should be an int
* itemInSession should be an int
* userid should be int

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO user_lib (userid, sessionId, itemInSession, artist, song, first, last)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### Verifying result output from Query 2

In [16]:
query = "SELECT artist, song, first, last from user_lib WHERE userid = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first, row.last)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### The table structure for Query 3:

As per the query only first name, last name of users is required who listened toa song.
If, Partiotion by song (Primary Key) that would override the N users who listened to a song & also it will not be unique.
So, Using (song, userid) as composite Primary Key would be a ideal case, as userid is unique and the data will be partitioned by song. 

Further, the query needs to fetch first, last name of all users who listened to a song. 

In [18]:
query = "CREATE TABLE IF NOT EXISTS user_song_lib"
query = query + "(song text, userid int, first text, last text, sessionId int, itemInSession int, PRIMARY KEY(song, userid))"
try: 
    session.execute(query)
except Exception as e:
    print(e)

### Inserting Data into Table:

As the base data is of string type basic transformations were applied while inserting these 3 fields:

* sessionId should be an int
* itemInSession should be an int
* userid should be int

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:

        query = "INSERT INTO user_song_lib (song, userid, first, last, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]),line[1], line[4], int(line[8]), int(line[3])))

In [20]:
query = "SELECT first, last from user_song_lib WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first, row.last)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
query = "drop table sessions_table"
rows = session.execute(query)

In [22]:
query = "drop table user_lib"
rows = session.execute(query)

In [23]:
query = "drop table user_song_lib"
rows = session.execute(query)

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()